### Multi-turn conversation with OpenAI API

In [1]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

# Define a variable for the chat completion
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Teach me how to do a backflip"}
]

# Initial response
response = client.chat.completions.create(
    model='gpt-4',
    messages=messages
)
ai_response = response.choices[0].message.content
print("AI Response:", ai_response)

messages.append({"role": "assistant", "content": ai_response})

# Allow for two more conversation rounds
for _ in range(2):
    user_input = input("You:")
    messages.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model='gpt-4',
        messages=messages
    )
    ai_response = response.choices[0].message.content
    print("AI Response:", f"{ai_response}\n\n")
    messages.append({"role": "assistant", "content": ai_response})

print("Conversation ended.")


AI Response: Performing a backflip is quite challenging and potentially dangerous if not done correctly. It involves a lot of strength and balance, as well as confidence. Therefore, it's advised to learn this with the help of a professional coach in a safe environment (gym or a training facility with mats or a foam pit).

However, I can give you a general explanation of the steps involved:

1. **Warm Up:** Always start with a good warmup to protect against injury. This includes a few minutes of light cardio followed by stretching.

2. **The Takeoff:** Stand tall, feet shoulder-width apart. Then swing your arms down by your sides and bend your knees into a squat position. Jump off the ground with both feet at the same time, and at the peak of your jump, tuck your knees in toward your chest. 

3. **Mid-air Rotation:** As you jump, swing your arms toward the sky, which will provide the rotation you need for the backflip. Also tuck your knees in tightly to your chest and try to keep your h

### Multi-turn conversation with Langchain memory module

In [2]:
from langchain.chains.conversation.memory import ConversationBufferMemory, ConversationSummaryMemory, ConversationBufferWindowMemory
from langchain.memory import ConversationTokenBufferMemory
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

`ConversationBuffer Memory`

In [3]:
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Define the prompt template
prompt = PromptTemplate(
    input_variables=["history", "input"],  # Include 'history'
    template="The following is a conversation between Me and AI:\n\n{history}\n\nMe: {input}\nAI:"
)

# Set up memory
memory = ConversationBufferMemory(
    return_messages=False,
    ai_prefix='AI',
    human_prefix='Me'
)

# Instantiate the LLMChain
conversation = ConversationChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

# Start the conversation
response = conversation.predict(input="Teach me how to do a backflip in only 100 words")
print(response)


C:\Users\okwy_\AppData\Local\Temp\ipykernel_32524\3383638983.py:16: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(
C:\Users\okwy_\AppData\Local\Temp\ipykernel_32524\3383638983.py:23: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between Me and AI:



Me: Teach me how to do a backflip in only 100 words
AI:

> Finished chain.
"Ahaha, a backflip in 100 words? That's a challenge! Okay, here's a concise guide:

1. Find a safe, open space with a soft landing surface.
2. Stand with your feet shoulder-width apart, facing the direction you want to flip.
3. Bend your knees, swing your arms back, and jump off the ground with both feet.
4. As you're in the air, tuck your knees to your chest and extend your body.
5. Rotate your body 360 degrees, keeping your arms and legs straight.
6. Extend your body again, and land on both feet with your knees bent to absorb the impact.

Remember, practice makes perfect! Start with a smaller flip and work your way up."


In [4]:
response = conversation.predict(input="What did I ask you before")
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between Me and AI:

Me: Teach me how to do a backflip in only 100 words
AI: "Ahaha, a backflip in 100 words? That's a challenge! Okay, here's a concise guide:

1. Find a safe, open space with a soft landing surface.
2. Stand with your feet shoulder-width apart, facing the direction you want to flip.
3. Bend your knees, swing your arms back, and jump off the ground with both feet.
4. As you're in the air, tuck your knees to your chest and extend your body.
5. Rotate your body 360 degrees, keeping your arms and legs straight.
6. Extend your body again, and land on both feet with your knees bent to absorb the impact.

Remember, practice makes perfect! Start with a smaller flip and work your way up."

Me: What did I ask you before
AI:

> Finished chain.
You asked me to teach you how to do a backflip in only 100 words.


In [5]:
print(memory.buffer)

Me: Teach me how to do a backflip in only 100 words
AI: "Ahaha, a backflip in 100 words? That's a challenge! Okay, here's a concise guide:

1. Find a safe, open space with a soft landing surface.
2. Stand with your feet shoulder-width apart, facing the direction you want to flip.
3. Bend your knees, swing your arms back, and jump off the ground with both feet.
4. As you're in the air, tuck your knees to your chest and extend your body.
5. Rotate your body 360 degrees, keeping your arms and legs straight.
6. Extend your body again, and land on both feet with your knees bent to absorb the impact.

Remember, practice makes perfect! Start with a smaller flip and work your way up."
Me: What did I ask you before
AI: You asked me to teach you how to do a backflip in only 100 words.


`Advanced: LangGraph`

In [6]:
from typing import Annotated

from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from dotenv import load_dotenv
load_dotenv()

memory = MemorySaver()

class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile(checkpointer=memory)

In [7]:
config = {"configurable": {"thread_id": "4"}}

In [8]:
user_input = "Teach me how to do a backflip"

events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Teach me how to do a backflip
================================== Ai Message ==================================

The backflip! It's an impressive and exciting move that requires practice, patience, and proper technique. I'll guide you through the steps to help you master the backflip. Please note that it's essential to warm up before attempting any new exercise or movement, and to start with a lower height and gradually increase it as you build confidence and skill.

**Preparation**

1. **Find a safe space**: Look for a large, open area with a soft surface, such as a grassy field, a gymnastics mat, or a spring floor. Avoid hard surfaces like concrete or wood, as they can cause injury.
2. **Warm up**: Do some light cardio and stretching exercises to get your body ready for the movement. Focus on your legs, back, and core muscles.
3. **Get comfortable with the movement**: Start by practicing the motion witho

In [9]:
user_input = "What was the last question I asked you?"

events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What was the last question I asked you?
================================== Ai Message ==================================

Your last question was "Teach me how to do a backflip".
